# Create NIHR Awards from NIHR Open Data

Creates NIHR awards from the NIHR Open Data API (UK National Institute for Health and Care Research). ~10K projects.

**Prerequisites:**
- Run `scripts/local/nihr_to_s3.py` to download and upload the data first.

**Data source:** https://nihr.opendatasoft.com/explore/dataset/nihr-summary-view/  
**S3 location:** `s3a://openalex-ingest/awards/nihr/nihr_projects.parquet`

**NIHR funder:**
- funder_id: 4320319990
- display_name: "National Institute for Health and Care Research"
- ROR: https://ror.org/0187kwz08
- DOI: 10.13039/501100000272

**NIHR Programmes:**
- NIHR Fellowships
- Health Technology Assessment
- Research for Patient Benefit
- Health and Social Care Delivery Research
- Policy Research Programme
- And many more...

## Step 1: Create Staging Table from S3

In [ ]:
%sql
-- Create the staging table from S3 parquet
CREATE OR REPLACE TABLE openalex.awards.nihr_raw
USING delta
AS
SELECT
    *,
    current_timestamp() as databricks_ingested_at
FROM parquet.`s3a://openalex-ingest/awards/nihr/nihr_projects.parquet`;

In [ ]:
%sql
-- Check row count (should be ~10K)
SELECT COUNT(*) as total_projects FROM openalex.awards.nihr_raw;

In [ ]:
%sql
-- Sample the raw data
SELECT * FROM openalex.awards.nihr_raw LIMIT 5;

In [ ]:
%sql
-- Check column names
DESCRIBE openalex.awards.nihr_raw;

## Step 2: Create NIHR Awards Table

In [ ]:
%sql
CREATE OR REPLACE TABLE openalex.awards.nihr_awards
USING delta
AS
WITH
-- Get NIHR funder from OpenAlex by explicit funder_id
nihr_funder AS (
    SELECT
        funder_id,
        display_name,
        ror_id,
        doi
    FROM openalex.common.funder
    WHERE funder_id = 4320319990  -- National Institute for Health and Care Research
),

awards_transformed AS (
    SELECT
        -- Generate unique ID using xxhash64 of funder_id:project_id
        abs(xxhash64(CONCAT(f.funder_id, ':', LOWER(g.project_id)))) % 9000000000 as id,

        -- Display name = project title
        g.title as display_name,

        -- Description = plain English abstract (preferred) or scientific abstract
        COALESCE(g.plain_english_abstract, g.scientific_abstract) as description,

        -- Funder info
        f.funder_id,
        g.project_id as funder_award_id,

        -- Amount in GBP
        TRY_CAST(g.award_amount AS DOUBLE) as amount,
        'GBP' as currency,

        -- Funder struct
        struct(
            CONCAT('https://openalex.org/F', f.funder_id) as id,
            f.display_name,
            f.ror_id,
            f.doi
        ) as funder,

        -- Funding type - map NIHR programmes to types
        CASE
            WHEN LOWER(g.programme) LIKE '%fellowship%' THEN 'fellowship'
            WHEN LOWER(g.programme) LIKE '%senior investigator%' THEN 'fellowship'
            WHEN LOWER(g.programme) LIKE '%academic%' THEN 'fellowship'
            WHEN LOWER(g.programme) LIKE '%bridging%' THEN 'fellowship'
            WHEN LOWER(g.programme) LIKE '%health technology%' THEN 'research'
            WHEN LOWER(g.programme) LIKE '%research for patient%' THEN 'research'
            WHEN LOWER(g.programme) LIKE '%programme grant%' THEN 'research'
            WHEN LOWER(g.programme) LIKE '%efficacy%' THEN 'research'
            WHEN LOWER(g.programme) LIKE '%public health%' THEN 'research'
            WHEN LOWER(g.programme) LIKE '%policy%' THEN 'research'
            WHEN LOWER(g.programme) LIKE '%global health%' THEN 'research'
            WHEN LOWER(g.programme) LIKE '%delivery%' THEN 'research'
            WHEN LOWER(g.programme) LIKE '%invention%' THEN 'research'
            WHEN LOWER(g.programme) LIKE '%infrastructure%' THEN 'infrastructure'
            ELSE 'grant'
        END as funding_type,

        -- Funder scheme = programme name
        g.programme as funder_scheme,

        -- Provenance
        'nihr' as provenance,

        -- Dates (stored as strings in YYYY-MM-DD format)
        TRY_TO_DATE(g.start_date, 'yyyy-MM-dd') as start_date,
        TRY_TO_DATE(g.end_date, 'yyyy-MM-dd') as end_date,
        YEAR(TRY_TO_DATE(g.start_date, 'yyyy-MM-dd')) as start_year,
        YEAR(TRY_TO_DATE(g.end_date, 'yyyy-MM-dd')) as end_year,

        -- Lead investigator
        CASE
            WHEN g.pi_family_name IS NOT NULL THEN
                struct(
                    g.pi_given_name as given_name,
                    g.pi_family_name as family_name,
                    g.orcid as orcid,
                    CAST(NULL AS DATE) as role_start,
                    struct(
                        g.contracted_organisation as name,
                        COALESCE(IF(size(g.institution_country) > 0, g.institution_country[0], NULL), 'United Kingdom') as country,
                        CAST(NULL AS ARRAY<STRUCT<id:STRING, type:STRING, asserted_by:STRING>>) as ids
                    ) as affiliation
                )
            ELSE NULL
        END as lead_investigator,

        -- Co-lead and other investigators (not available in NIHR Open Data)
        CAST(NULL AS STRUCT<
            given_name:STRING,
            family_name:STRING,
            orcid:STRING,
            role_start:DATE,
            affiliation:STRUCT<name:STRING, country:STRING, ids:ARRAY<STRUCT<id:STRING, type:STRING, asserted_by:STRING>>>
        >) as co_lead_investigator,

        CAST(NULL AS ARRAY<STRUCT<
            given_name:STRING,
            family_name:STRING,
            orcid:STRING,
            role_start:DATE,
            affiliation:STRUCT<name:STRING, country:STRING, ids:ARRAY<STRUCT<id:STRING, type:STRING, asserted_by:STRING>>>
        >>) as investigators,

        -- Landing page URL from data
        g.landing_page_url as landing_page_url,

        -- No DOI for NIHR grants typically
        CAST(NULL AS STRING) as doi,

        -- Works API URL
        concat('https://api.openalex.org/works?filter=awards.id:G', abs(xxhash64(CONCAT(f.funder_id, ':', LOWER(g.project_id)))) % 9000000000) as works_api_url,

        -- Timestamps
        current_timestamp() as created_date,
        current_timestamp() as updated_date

    FROM openalex.awards.nihr_raw g
    CROSS JOIN nihr_funder f
    WHERE g.project_id IS NOT NULL
      AND TRIM(g.project_id) != ''
)

SELECT * FROM awards_transformed;

In [ ]:
%sql
-- Insert into openalex_awards_raw with priority
INSERT INTO openalex.awards.openalex_awards_raw
SELECT
    id,
    display_name,
    description,
    funder_id,
    funder_award_id,
    amount,
    currency,
    funder,
    funding_type,
    funder_scheme,
    provenance,
    start_date,
    end_date,
    start_year,
    end_year,
    lead_investigator,
    co_lead_investigator,
    investigators,
    landing_page_url,
    doi,
    works_api_url,
    created_date,
    updated_date,
    13 as priority
FROM openalex.awards.nihr_awards;

## Verification Queries

In [ ]:
%sql
-- Check row count (should be ~10K)
SELECT COUNT(*) as total_nihr_awards FROM openalex.awards.nihr_awards;

In [ ]:
%sql
-- Sample the data
SELECT 
    id,
    display_name,
    funder_award_id,
    funder_scheme,
    funding_type,
    amount,
    currency,
    start_date,
    end_date,
    lead_investigator
FROM openalex.awards.nihr_awards 
LIMIT 10;

In [ ]:
%sql
-- Check funder distribution (should all be NIHR)
SELECT funder.display_name, COUNT(*) as cnt
FROM openalex.awards.nihr_awards
GROUP BY funder.display_name
ORDER BY cnt DESC;

In [ ]:
%sql
-- Check funding_type distribution
SELECT funding_type, COUNT(*) as cnt
FROM openalex.awards.nihr_awards
GROUP BY funding_type
ORDER BY cnt DESC;

In [ ]:
%sql
-- Check funder_scheme distribution (top 20 programmes)
SELECT funder_scheme, COUNT(*) as cnt,
       ROUND(SUM(amount)/1e6, 2) as total_amount_millions_gbp
FROM openalex.awards.nihr_awards
WHERE funder_scheme IS NOT NULL
GROUP BY funder_scheme
ORDER BY cnt DESC
LIMIT 20;

In [ ]:
%sql
-- Check data completeness
SELECT
    COUNT(*) as total,
    COUNT(display_name) as has_title,
    COUNT(description) as has_abstract,
    COUNT(amount) as has_amount,
    COUNT(start_date) as has_start_date,
    COUNT(end_date) as has_end_date,
    COUNT(lead_investigator) as has_pi,
    SUM(CASE WHEN lead_investigator.orcid IS NOT NULL THEN 1 ELSE 0 END) as has_orcid,
    ROUND(COUNT(lead_investigator) * 100.0 / COUNT(*), 1) as pct_with_pi,
    ROUND(COUNT(amount) * 100.0 / COUNT(*), 1) as pct_with_amount,
    ROUND(SUM(amount)/1e9, 2) as total_amount_billions_gbp
FROM openalex.awards.nihr_awards;

In [ ]:
%sql
-- Check year distribution
SELECT start_year, COUNT(*) as cnt,
       ROUND(SUM(amount)/1e6, 2) as total_amount_millions_gbp
FROM openalex.awards.nihr_awards
WHERE start_year IS NOT NULL
GROUP BY start_year
ORDER BY start_year DESC
LIMIT 20;

In [ ]:
%sql
-- Check lead institutions (top 20)
SELECT 
    lead_investigator.affiliation.name as institution,
    COUNT(*) as grant_count,
    ROUND(SUM(amount)/1e6, 1) as total_amount_millions_gbp
FROM openalex.awards.nihr_awards
WHERE lead_investigator.affiliation.name IS NOT NULL
GROUP BY lead_investigator.affiliation.name
ORDER BY total_amount_millions_gbp DESC
LIMIT 20;